In [ ]:
# detect_gender
# import libraries
import cv2
import cvlib as cv
import numpy as np

image_path = 'img/lena.jpg'
im = cv2.imread(image_path) # 이미지 읽기


# detect faces (얼굴 검출)
faces, confidences = cv.detect_face(im)

for face in faces:
    (startX,startY) = face[0],face[1]
    (endX,endY) = face[2],face[3]
    # draw rectangle over face
    cv2.rectangle(im, (startX,startY), (endX,endY), (0,255,0), 2) # 검출된 얼굴 위에 박스 그리기
    face_crop = np.copy(im[startY:endY, startX:endX])
    
    # gender detection (성별 검출)
    (label, confidence) = cv.detect_gender(face_crop)
    
    print(confidence)
    print(label)
    
    idx = np.argmax(confidence)
    label = label[idx]

    label = "{}: {:.2f}%".format(label, confidence[idx] * 100)

    Y = startY - 10 if startY - 10 > 10 else startY + 10

    cv2.putText(im, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (0, 255, 0), 2) # 박스 위에 남자인지 여자인지 라벨과 확률 쓰기
    

cv2.imshow('result', im)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# face_detectionusingHaarCascades

import numpy as np
import cv2
from matplotlib import pyplot as plt
face_cascade = cv2.CascadeClassifier('data/haarcascades/haarcascade_frontface.xml')
eye_cascade = cv2.CascadeClassifier('data/haarcascades/haarcascade_eye.xml')
img = cv2.imread('img/lena.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 입력 이미지에서 얼굴을 검출하는 것입니다. 만약 얼굴을 발견하면, 
# 발견한 얼굴에 대한 위치를 Rect(x,y,w,h) 형태로 얻음
# 이 위치를 얻었다면, 얼굴에 대한 ROI를 만들고, 이 안에서 눈을 검출
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
cv2.imshow('img',img)
cv2.waitKey(0)

In [ ]:
# harris_corner_detector
'''
코너 검출
cv2.cornerHarris() for this purpose. Its arguments are :
img - Input image, it should be grayscale and float32 type.
blockSize - It is the size of neighbourhood considered for corner detection
ksize - Aperture parameter of Sobel derivative used.
k - Harris detector free parameter in the equation.
'''
import cv2
import numpy as np

img = cv2.imread('img/board.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)

#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]

cv2.imshow('dst',img)
if cv2.waitKey(0) & 0xff == 27:
    cv2.destroyAllWindows()

In [ ]:
# hist_backprojection
import cv2
import numpy as np
roi = cv2.imread('img/test_1.jpg')
hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
target = cv2.imread('img/test.jpg')
hsvt = cv2.cvtColor(target,cv2.COLOR_BGR2HSV)
# calculating object histogram
roihist = cv2.calcHist([hsv],[0, 1], None, [180, 256], [0, 180, 0, 256] )
# normalize histogram and apply backprojection
cv2.normalize(roihist,roihist,0,255,cv2.NORM_MINMAX)
dst = cv2.calcBackProject([hsvt],[0,1],roihist,[0,180,0,256],1)
# 2차원 히스토그램을 응용하여 이미지에서 원하는 객체만을 추출해 내는 방법
# Now convolute with circular disc
disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
cv2.filter2D(dst,-1,disc,dst)
# threshold
ret,thresh = cv2.threshold(dst,50,255,0)
# threshold and binary AND
thresh = cv2.merge((thresh,thresh,thresh))
res = cv2.bitwise_and(target,thresh)
res = np.vstack((thresh,res))
cv2.imshow('result', res)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# histogram_calcHist
'''
# Histogram은 이미지의 밝기의 분포를 그래프로 표현
# 히스토그램을 이용하면 이미지의 전체의 밝기 분포와 채도(색의 밝고 어두움)를 알 수 있음
cv2.calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate]])
Parameters:	
image – 분석대상 이미지(uint8 or float32 type). Array형태.
channels – 분석 채널(X축의 대상). 이미지가 graysacle이면 [0], color 이미지이면 [0],[0,1] 형태(1 : Blue, 2: Green, 3: Red)
mask – 이미지의 분석영역. None이면 전체 영역.
histSize – BINS 값. [256]
ranges – Range값. [0,256]
'''
import cv2
import numpy as np
import random
from matplotlib import pyplot as plt

img1 = cv2.imread('img/flower.jpg',0)
img2 = cv2.imread('img/flower2.png',0)

hist1 = cv2.calcHist([img1],[0],None,[256],[0,256])
hist2 = cv2.calcHist([img2],[0],None,[256],[0,256])

plt.subplot(221),plt.imshow(img1,'gray'),plt.title('RED Line')
plt.subplot(222),plt.imshow(img2,'gray'),plt.title('YELLOW Line')
plt.subplot(223),plt.plot(hist1,color='r'),plt.plot(hist2,color='y')
plt.xlim([0,256])
plt.show()

In [ ]:
# histogram_clahe
'''
지금까지의 처리는 이미지의 전체적인 부분에 균일화를 적용
하지만 일반적인 이미지는 밝은 부분과 어두운 부분이 섞여 있기 때문에 
전체에 적용하는 것은 그렇게 유용하지 않음
contrast limit라는 값을 적용하여 이 값을 넘어가는 경우는 그 영역은 다른 영역에 균일하게 배분하여 적용
'''
import cv2
import numpy as np
from matplotlib import pyplot as plt


img = cv2.imread('img/hist_test2.png',0)

# contrast limit가 2이고 title의 size는 8X8
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
img2 = clahe.apply(img)

img = cv2.resize(img,(400,400))
img2 = cv2.resize(img2,(400,400))

dst = np.hstack((img, img2))
cv2.imshow('img',dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# histogram_equalization
'''
히스토그램 균일화(Histogram Equalization)에 대해서 알 수 있고, 
이것을 이용하여 이미지의 contrast를 향상
이미지의 히스토그램이 특정영역에 너무 집중되어 있으면 contrast가 낮아 좋은 이미지가 아님
전체 영역에 골고루 분포가 되어 있을 때 좋은 이미지
특정 영역에 집중되어 있는 분포를 골고루 분포하도록 하는 작업을 Histogram Equalization
'''
import cv2
import numpy as np
from matplotlib import pyplot as plt


img = cv2.imread('img/hist_test.png',0)

# OpenCV의 Equaliztion함수
img2 = cv2.equalizeHist(img)
img = cv2.resize(img,(400,400))
img2 = cv2.resize(img2,(400,400))

dst = np.hstack((img, img2))
cv2.imshow('img',dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# hough_circle_transform
'''
이미지에서 원을 찾을 수 있는 허프변환
cv2.HoughCircles(image, method, dp, minDist[, circles[, param1[, param2[, minRadius[, maxRadius]]]]]) → circles
Parameters:	
image – 8-bit single-channel image. grayscale image.
method – 검출 방법. 현재는 HOUGH_GRADIENT가 있음.
dp – dp=1이면 Input Image와 동일한 해상도.
minDist – 검출한 원의 중심과의 최소거리. 값이 작으면 원이 아닌 것들도 검출이 되고, 너무 크면 원을 놓칠 수 있음.
param1 – 내부적으로 사용하는 canny edge 검출기에 전달되는 Paramter
param2 – 이 값이 작을 수록 오류가 높아짐. 크면 검출률이 낮아짐.
minRadius – 원의 최소 반지름.
maxRadius – 원의 최대 반지름.
'''
import cv2
import numpy as np

img = cv2.imread('img/logo.jpg',0)
img = cv2.medianBlur(img,5)
cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)

circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1, 20,param1=50,param2=25,minRadius=0, maxRadius=0)

circles = np.uint16(np.around(circles))

for i in circles[0,:]:
    cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
    cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)

cv2.imshow('img', cimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# hough_houghlinesP
'''
cv2.HoughLinesP(image, rho, theta, threshold, minLineLength, maxLineGap) → lines
Parameters:	
image – 8bit, single-channel binary image, canny edge를 선 적용.
rho – r 값의 범위 (0 ~ 1 실수)
theta – 𝜃 값의 범위(0 ~ 180 정수)
threshold – 만나는 점의 기준, 숫자가 작으면 많은 선이 검출되지만 정확도가 떨어지고, 숫자가 크면 정확도가 올라감.
minLineLength – 선의 최소 길이. 이 값보다 작으면 reject.
maxLineGap – 선과 선사이의 최대 허용간격. 이 값보다 작으며 reject.
'''
import cv2
import numpy as np

img = cv2.imread('img/dave.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,200,apertureSize = 3)
minLineLength = 100
maxLineGap = 0

lines = cv2.HoughLinesP(edges,1,np.pi/360,100,minLineLength,maxLineGap)

for i in range(len(lines)): #395
    for x1,y1,x2,y2 in lines[i]:
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),3)


cv2.imshow('img1',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# image_cv2.fastNlMeansDenoisingColored_denoising
'''
이미지에서 잡음 제거

cv2.fastNlMeansDenoising() – 그레이 이미지 하나에 대해서만 작동함
cv2.fastNlMeansDenoisingColored() – 칼라 이미지 하나에 대해서 작동함
cv2.fastNlMeansDenoisingMulti() – 짧은 시간 동안 찍힌 여러 개의 이미지에 대해서 작동하며 그레이 이미지여야 함
cv2.fastNlMeansDenoisingColoredMulti() – 짧은 시간 동안 찍한 여러 개의 이미지에 대해서 작동하며 
칼라 이미지에서 작동함
위 함수들의 공통 인자는 다음과 같습니다.
h : 필터 강도를 결정하는 인자. 더 높은 h 값이 잡음을 더 잘 제거하지만 잡음이 아닌 픽셀도 제거함(10이면 적당함)
hForColorComponents : h와 동일하지만, 칼라 이미지에 대해서만 사용됨(보통 h와 같음)
templateWindowSize : 홀수값이여야 함(7을 권장함)
searchWindowSize : 홀수값이여야 함(21을 권장함)
'''

import numpy as np
import cv2
from matplotlib import pyplot as plt
img = cv2.imread('img/pill.png')
dst = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
plt.subplot(121),plt.imshow(img)
plt.subplot(122),plt.imshow(dst)
plt.show()

In [ ]:
# image_thresholding

'''
cv2.threshold(src, thresh, maxval, type) → retval, dst
Parameters:	
src – input image로 single-channel 이미지.(grayscale 이미지)
thresh – 임계값
maxval – 임계값을 넘었을 때 적용할 value
type – thresholding type
'''

import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('img/gradient.png',0)
ret,thresh1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
ret,thresh2 = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
ret,thresh3 = cv2.threshold(img,127,255,cv2.THRESH_TRUNC)
ret,thresh4 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO)
ret,thresh5 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO_INV)

titles = ['Original Image','BINARY','BINARY_INV','TRUNC','TOZERO','TOZERO_INV']
images = [img, thresh1, thresh2, thresh3, thresh4, thresh5]

for i in range(6):
    plt.subplot(2,3,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])

plt.show()